**Cross-Encoder Re-Ranking**

Install sentence-transformers python library  and import 'CrossEncoder' from this library.

In [3]:
!pip install sentence-transformers
from sentence_transformers import CrossEncoder

In [ ]:
# place your code here

From typing  import List, Tuple

In [4]:
from typing import List, Tuple

Load the pre-trained cross-encoder model 'cross-encoder/ms-marco-MiniLM-L-6-v2' and save the model object to the 'model' variable. To load, use CrossEncoder() with two arguments: the first is the model name as string and the second argument is 'max_length=512'

In [5]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Use function:

In [6]:
def rerank_documents(
    query: str,  documents: List[str],  top_k: int = 10
) -> List[Tuple[int, str, float]]:
    """
    Re-rank documents based on relevance to query.
    Args:
        query: The search query
        documents: List of document texts from initial retrieval
        top_k: Number of top results to return
    Returns:
        List of (original_index, document, score) tuples, sorted by relevance
    """
    # Create query-document pairs for the cross-encoder
    # Each pair will be scored independently
    pairs = [[query, doc] for doc in documents]

    # Get relevance scores for all pairs
    # The model outputs a single score per pair
    scores = model.predict(pairs)

    # Combine with original indices and sort by score descending
    scored_docs = [
        (idx, doc, float(score))
        for idx, (doc, score) in enumerate(zip(documents, scores))
    ]
    scored_docs.sort(key=lambda x: x[2], reverse=True)

    return scored_docs[:top_k]

Simulate retrieval results: use the following data.

In [7]:
retrieved_docs = [
        "Kubernetes pod monitoring requires metrics collection from the kubelet.",
        "Docker containers can be monitored using cAdvisor metrics.",
        "To check pod status, use kubectl get pods command.",
        "Prometheus is commonly used for Kubernetes monitoring.",
        "Pod resource limits should be set in the deployment spec.",
        "OneUptime provides real-time Kubernetes monitoring dashboards.",
        "Container orchestration platforms need observability solutions.",
        "The kubectl logs command shows container output.",
    ]
query = "How do I monitor Kubernetes pods?"

Select a value for the k parameter and apply the "rerank_documents()" function to the data.
Save the returned object to the results variabl.

In [9]:
k = 5

results = rerank_documents(query, retrieved_docs, top_k=k)

Using the 'for idx, doc, score in results'    loop, print the values ​​of 'idx', 'doc' and 'score'

In [10]:
print("Re-ranked results:")
for idx, doc, score in results:
    print("Index:", idx)
    print("Document:", doc)
    print("Score:", score)
    print()


Re-ranked results:
Index: 0
Document: Kubernetes pod monitoring requires metrics collection from the kubelet.
Score: 6.1957926750183105

Index: 5
Document: OneUptime provides real-time Kubernetes monitoring dashboards.
Score: 2.8766822814941406

Index: 3
Document: Prometheus is commonly used for Kubernetes monitoring.
Score: 2.8236846923828125

Index: 2
Document: To check pod status, use kubectl get pods command.
Score: 0.14170357584953308

Index: 7
Document: The kubectl logs command shows container output.
Score: -8.542130470275879

